In [83]:
import pandas as pd
import numpy as np

In [84]:
df = pd.read_csv('dataemploi_clean.csv', encoding='latin-1')
df.head()

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities,Job Title Extracted,Country,Salary Category,Remote,Hybrid,Local,Category
0,sgs,clinical data analyst,"richardson, tx, united states",full time,entry-level,48000.0,"computer science,data quality,genetics,mathema...",without Facilities,data analyst,United States,Salaire minimum et sous-conditions,False,False,True,Data Science
1,ocorian,aml/cft & data analyst,"ebÃ¨ne, mauritius",full time,entry-level,48000.0,"agile,data management,finance,security",without Facilities,data analyst,Mauritius / Maurice,Salaire minimum et sous-conditions,False,False,True,Data Science
2,cricut,machine learning engineer,"south jordan, ut, united states",full time,mid-level,90000.0,"agile,architecture,aws,computer science,comput...",career development,machine learning engineer,United States,Salaire minimum et sous-conditions,False,False,True,Other
3,bosch group,application developer & data analyst,"nonantola, italy",full time,entry-level,48000.0,"engineering,industrial,oracle,power bi,r,r&d",without Facilities,data analyst,Italia,Salaire minimum et sous-conditions,False,False,True,Data Science
4,publicis groupe,data engineer full time (public sector) usa,"arlington, va, united states",full time,mid-level,108000.0,"aws,azure,computer science,consulting,dataflow...","flex hours,flex vacation,parental leave,unlimi...",data engineer,United States,Salaire minimum,False,False,True,Big Data


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2915 entries, 0 to 2914
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Company                    2915 non-null   object 
 1   Job Title                  2915 non-null   object 
 2   Location                   2915 non-null   object 
 3   Job Type                   2915 non-null   object 
 4   Experience level           2915 non-null   object 
 5   Salary                     2915 non-null   float64
 6   Requirment of the company  2915 non-null   object 
 7   Facilities                 2915 non-null   object 
 8   Job Title Extracted        2915 non-null   object 
 9   Country                    2915 non-null   object 
 10  Salary Category            2915 non-null   object 
 11  Remote                     2915 non-null   bool   
 12  Hybrid                     2915 non-null   bool   
 13  Local                      2915 non-null   bool 

In [86]:
category_mapping = {
    "AI": ["robotic scientist", "ai product manager", "ai consultant", "robotics engineer", "nlp engineer", "research assistant", "deep learning engineer", "artificial intelligence (ai) engineer"],
    "Data Science": ["data scientist", "data analyst", "machine learning scientist", "business intelligence analyst", "data security analyst", "data modeler", "data quality analyst", "data visualization specialist", "data governance analyst", "data insights analyst", "data privacy officer", "data strategy consultant", "data science manager"],
    "Big Data": ["data engineer", "data architect", "big data engineer", "data warehouse specialist", "data operations manager", "data product manager", "data integration specialist", "data mining specialist", "data analytics manager", "data migration specialist", "data governance manager"]
}

# Fonction pour mapper les titres de poste aux catégories
def categorize_job_title(job_title):
    for category, titles in category_mapping.items():
        if job_title.lower() in titles:
            return category
    return "Other"  


df['Category'] = df['Job Title Extracted'].apply(categorize_job_title)

df['Category']

0       Data Science
1       Data Science
2              Other
3       Data Science
4           Big Data
            ...     
2910        Big Data
2911    Data Science
2912        Big Data
2913    Data Science
2914    Data Science
Name: Category, Length: 2915, dtype: object

In [87]:
import altair as alt

offer_counts = df.groupby('Company').size().reset_index(name='Total Offers')

top_10_companies = offer_counts.sort_values(by='Total Offers', ascending=False).head(10)

chart = alt.Chart(top_10_companies).mark_bar().encode(
    x=alt.X('Total Offers:Q', title='Total Offers'),
    y=alt.Y('Company:N', sort='-x', title='Company'),
    color=alt.Color('Total Offers:Q', scale=alt.Scale(scheme='viridis'), title='Total Offers'),
    tooltip=['Company', 'Total Offers']
).properties(
    width=400,
    title='Top 10 Companies with the Highest Total Offers'
)

chart


alt.Chart(...)

In [88]:
top_10_companies

,Company,Total Offers
754,publicis groupe,111
157,bosch group,88
61,amazon.com,57
148,block,46
1017,visa,41
1011,verisk,31
655,nielseniq,30
362,experian,27
271,databricks,23
844,sia partners,22


In [105]:
import plotly.express as px

companies_per_country = df.groupby('Country')['Company'].nunique().reset_index()
companies_per_country.columns = ['Country', 'Number of Companies']

fig = px.choropleth(companies_per_country,
                    locations='Country',
                    locationmode='country names',
                    color='Number of Companies',
                    hover_name='Country',
                    color_continuous_scale='Viridis',
                    scope='world',
                    title='Number des offres par pays')

fig.show()


In [56]:
df.columns

Index(['Company', 'Job Title', 'Location', 'Job Type', 'Experience level',
       'Salary', 'Requirment of the company', 'Facilities',
       'Job Title Extracted', 'Country', 'Salary Category', 'Remote', 'Hybrid',
       'Local', 'Category'],
      dtype='object')

In [94]:
category_counts = df['Category'].value_counts()
category_counts

Data Science    1525
Big Data         884
Other            394
AI               112
Name: Category, dtype: int64

In [95]:
category_counts = df['Category'].value_counts()

category_counts_df = category_counts.reset_index()
category_counts_df.columns = ['Category', 'Count']

fig = px.bar(category_counts_df, x='Category', y='Count', labels={'Category': 'Category', 'Count': 'Count'},
             title='Count of Each Category')
fig.update_layout(xaxis_tickangle=-45)

fig.show()


In [59]:
category_country_counts = df.groupby('Category')['Country'].value_counts()
top_10_countries_per_category = category_country_counts.groupby('Category').nlargest(10).reset_index(level=0, drop=True)
top_10_countries_per_category

Category      Country                     
AI            United States                    38
              India                            11
              United Kingdom                    8
              France                            8
              Canada                            6
              The Netherlands                   5
              Vietnam                           3
              Thailand                          3
              Deutschland                       3
              Poland                            2
Big Data      United States                   327
              India                            86
              United Kingdom                   64
              France                           42
              Canada                           35
              Deutschland                      26
              Portugal                         18
              Poland                           12
              spain                            12
       

In [120]:
# Filtrez la catégorie "Other" des données
filtered_data = top_10_countries_per_category[top_10_countries_per_category.index.get_level_values('category') != 'Other']

filtered_data = filtered_data.reset_index()

fig = px.bar(filtered_data, x='Country', y='count', color='Category',
             labels={'Country': 'Country', 'count': 'Count', 'category': 'Category'},
             title='Top 10 Countries for Each Category (Excluding "Other")')

fig.show()


NameError: name 'top_10_countries_per_category' is not defined

In [61]:
df['Job Type'].value_counts()

Job Type
full time    2879
part time       4
Name: count, dtype: int64

In [62]:
mean_salary_by_experience_country = df.groupby(['Experience level', 'Country'])['Salary'].mean().reset_index()
mean_salary_by_experience_country = mean_salary_by_experience_country.sort_values(by='Salary', ascending=False)

top_10_senior_level = mean_salary_by_experience_country[mean_salary_by_experience_country['Experience level'] == 'senior-level'].head(10)
top_10_mid_level = mean_salary_by_experience_country[mean_salary_by_experience_country['Experience level'] == 'mid-level'].head(10)
top_10_entry_level = mean_salary_by_experience_country[mean_salary_by_experience_country['Experience level'] == 'entry-level'].head(10)

colors = {
    'senior-level': 'rgb(47,79,79)',
    'mid-level': 'rgb(119,136,153)',
    'entry-level': 'rgb(105,105,105)'
}

fig1 = px.bar(top_10_senior_level, x='Country', y='Salary', color='Experience level',
              labels={'Country': 'Country', 'Salary': 'Average Salary'},
              title='Top 10 Countries for Senior-level Experience', color_discrete_map=colors)
fig2 = px.bar(top_10_mid_level, x='Country', y='Salary', color='Experience level',
              labels={'Country': 'Country', 'Salary': 'Average Salary'},
              title='Top 10 Countries for Mid-level Experience', color_discrete_map=colors)
fig3 = px.bar(top_10_entry_level, x='Country', y='Salary', color='Experience level',
              labels={'Country': 'Country', 'Salary': 'Average Salary'},
              title='Top 10 Countries for Entry-level Experience', color_discrete_map=colors)

fig1.show()
fig2.show()
fig3.show()


In [63]:
total_job_opportunities = df['Experience level'].value_counts()

fig = px.pie(total_job_opportunities, names=total_job_opportunities.index, values=total_job_opportunities.values,
             title='Total Job Opportunities by Experience Level')
fig.show()


In [64]:
mean_salary_by_experience = df.groupby('Experience level')['Salary'].mean().reset_index()
fig = px.pie(mean_salary_by_experience, names='Experience level', values='Salary',
             title='Mean Salary by Experience Level')

fig.show()


In [65]:
import plotly.graph_objects as go

# Créer un tableau croisé (crosstab) entre 'Experience level' and 'Remote', 'Hybrid', 'Local'
crosstab_df = pd.crosstab(df['Experience level'], [df['Remote'], df['Hybrid'], df['Local']])

fig = go.Figure()

for remote in [True, False]:
    for hybrid in [True, False]:
        for local in [True, False]:
            if (remote, hybrid, local) in crosstab_df.columns:
                trace_name = f"Remote={remote}, Hybrid={hybrid}, Local={local}"
                fig.add_trace(go.Bar(x=crosstab_df.index,
                                     y=crosstab_df[(remote, hybrid, local)],
                                     name=trace_name))

fig.update_layout(title='Experience Level Analysis by Work Types',
                  xaxis_title='Experience Level',
                  yaxis_title='Count',
                  barmode='stack')

fig.show()


In [66]:
crosstab_df = pd.crosstab(df['Experience level'], df['Job Type'])

fig = go.Figure()

for job_type in crosstab_df.columns:
    fig.add_trace(go.Bar(x=crosstab_df.index,
                         y=crosstab_df[job_type],
                         name=job_type))

fig.update_layout(title='Experience Level Analysis by Job Type',
                  xaxis_title='Experience Level',
                  yaxis_title='Count',
                  barmode='stack')

fig.show()



In [67]:

top_10_entry_level = df[df['Experience level'] == 'entry-level']['Job Title Extracted'].value_counts().head(10)
top_10_mid_level = df[df['Experience level'] == 'mid-level']['Job Title Extracted'].value_counts().head(10)
top_10_senior_level = df[df['Experience level'] == 'senior-level']['Job Title Extracted'].value_counts().head(10)

colors = {
    'senior-level': 'rgb(47, 79, 79)',
    'mid-level': 'rgb(119, 136, 153)',
    'entry-level': 'rgb(105, 105, 105)'
}

fig1 = px.bar(top_10_entry_level, x=top_10_entry_level.index, y=top_10_entry_level.values,
              labels={'x': 'Job Title', 'y': 'Count'}, title='Top 10 Job Title Extracted for Entry-level',
              color_discrete_sequence=[colors['entry-level']] * len(top_10_entry_level))
fig2 = px.bar(top_10_mid_level, x=top_10_mid_level.index, y=top_10_mid_level.values,
              labels={'x': 'Job Title', 'y': 'Count'}, title='Top 10 Job Title Extracted for Mid-level',
              color_discrete_sequence=[colors['mid-level']] * len(top_10_mid_level))
fig3 = px.bar(top_10_senior_level, x=top_10_senior_level.index, y=top_10_senior_level.values,
              labels={'x': 'Job Title', 'y': 'Count'}, title='Top 10 Job Title Extracted for Senior-level',
              color_discrete_sequence=[colors['senior-level']] * len(top_10_senior_level))

fig3.show()
fig2.show()
fig1.show()


In [100]:
grouped_df = df.groupby(['Category'])['Job Title Extracted'].unique().reset_index()

grouped_df = df.groupby(['Category'])['Job Title Extracted'].apply(list).reset_index()

grouped_df = df.groupby(['Category'])['Job Title Extracted'].apply(', '.join).reset_index()


In [101]:
grouped_df

,Category,Job Title Extracted
0,AI,"artificial intelligence (ai) engineer, ai prod..."
1,Big Data,"data engineer, data engineer, data engineer, d..."
2,Data Science,"data analyst, data analyst, data analyst, data..."
3,Other,"machine learning engineer, bi developer, machi..."


In [102]:
fig = px.box(df, x='Category', y='Salary', color='Experience level',
             title='Boxplot of Salary for each Category colored by Experience level')
fig.show()


In [103]:
Job_Title_Extracted_df['Requirment of the company'].value_counts().head(20)

NameError: name 'Job_Title_Extracted_df' is not defined

In [99]:
top_20_requirements = Job_Title_Extracted_df['Requirment of the company'].value_counts().head(20)

top_20_df = pd.DataFrame({'Requirements': top_20_requirements.index, 'Count': top_20_requirements.values})

fig = px.bar(top_20_df, x='Requirements', y='Count', labels={'Requirements': 'Requirements', 'Count': 'Count'},
             title='Top 20 Requirements from Requirment of the company')
fig.update_layout(xaxis_tickangle=-45)

fig.show()


NameError: name 'Job_Title_Extracted_df' is not defined

In [98]:
requirements_list = df['Requirment of the company'].str.split(',')

# Créer un nouveau DataFrame avec Requirement of the company
requirements_by_category_exploded = pd.DataFrame({
    'Category': df['Category'].repeat(requirements_list.str.len()),
    'Requirement of the company': [item for sublist in requirements_list for item in sublist]
})

requirements_by_category_exploded = requirements_by_category_exploded.reset_index(drop=True)

requirements_by_category_exploded


,Category,Requirement of the company
0,Data Science,computer science
1,Data Science,data quality
2,Data Science,genetics
3,Data Science,mathematics
4,Data Science,sas
...,...,...
16931,Data Science,data analytics
16932,Data Science,data mining
16933,Data Science,economics
16934,Data Science,engineering


In [74]:
ai_category_df = requirements_by_category_exploded[requirements_by_category_exploded['Category'] == 'AI']
top_10_requirements_ai = ai_category_df['Requirement of the company'].value_counts().head(10)

ds_category_df = requirements_by_category_exploded[requirements_by_category_exploded['Category'] == 'Data Science']
top_10_requirements_DS = ai_category_df['Requirement of the company'].value_counts().head(10)

bd_category_df = requirements_by_category_exploded[requirements_by_category_exploded['Category'] == 'Big Data']
top_10_requirements_BD = ai_category_df['Requirement of the company'].value_counts().head(10)


In [75]:
top_10_requirements_BD

Requirement of the company
engineering         53
computer science    44
machine learning    35
architecture        34
aws                 17
computer vision     17
agile               16
deep learning       16
apis                15
python              14
Name: count, dtype: int64

In [76]:
fig1 = go.Figure(data=[go.Bar(x=top_10_requirements_ai.index, y=top_10_requirements_ai.values, marker_color='rgb(135,206,235)')])
fig2 = go.Figure(data=[go.Bar(x=top_10_requirements_DS.index, y=top_10_requirements_DS.values, marker_color='rgb(70,130,180)')])
fig3 = go.Figure(data=[go.Bar(x=top_10_requirements_BD.index, y=top_10_requirements_BD.values, marker_color='rgb(25,25,112)')])

fig1.update_layout(title='Top 10 Requirements for the "AI" Category',
                   xaxis_title='Requirement',
                   yaxis_title='Count')

fig2.update_layout(title='Top 10 Requirements for the "Data Science" Category',
                   xaxis_title='Requirement',
                   yaxis_title='Count')

fig3.update_layout(title='Top 10 Requirements for the "Big Data" Category',
                   xaxis_title='Requirement',
                   yaxis_title='Count')

fig1.show()
fig2.show()
fig3.show()


In [77]:
# Supprimer les lignes avec des noms d'établissement vides
Facilities_by_category_exploded = Facilities_by_category_exploded.drop(Facilities_by_category_exploded[Facilities_by_category_exploded['Facilities'] == ''].index)

ai_category_df = Facilities_by_category_exploded[Facilities_by_category_exploded['Category'] == 'AI']
top_10_facilities_ai = ai_category_df['Facilities'].value_counts().head(10)

top_10_facilities_ai

NameError: name 'Facilities_by_category_exploded' is not defined

In [78]:
facilities_list = df['Facilities'].str.split(',')

#  Créer un nouveau DataFrame avec facilities
Facilities_by_category_exploded = pd.DataFrame({
    'Category': df['Category'].repeat(facilities_list.str.len()),
    'Facilities': [item for sublist in facilities_list for item in sublist]
})

Facilities_by_category_exploded = Facilities_by_category_exploded.reset_index(drop=True)


In [104]:
ai_category_df = Facilities_by_category_exploded[Facilities_by_category_exploded['Category'] == 'AI']
top_10_Facilities_ai = ai_category_df['Facilities'].value_counts().head(10)

ds_category_df = Facilities_by_category_exploded[Facilities_by_category_exploded['Category'] == 'Data Science']
top_10_Facilities_DS = ai_category_df['Facilities'].value_counts().head(10)

bd_category_df = Facilities_by_category_exploded[Facilities_by_category_exploded['Category'] == 'Big Data']
top_10_Facilities_BD = ai_category_df['Facilities'].value_counts().head(10)

NameError: name 'Facilities_by_category_exploded' is not defined

In [80]:
fig1 = go.Figure(data=[go.Bar(x=top_10_Facilities_ai.index, y=top_10_Facilities_ai.values, marker_color='rgb(135,206,235)')])
fig2 = go.Figure(data=[go.Bar(x=top_10_Facilities_DS.index, y=top_10_Facilities_DS.values, marker_color='rgb(70,130,180)')])
fig3 = go.Figure(data=[go.Bar(x=top_10_Facilities_BD.index, y=top_10_Facilities_BD.values, marker_color='rgb(25,25,112)')])

fig1.update_layout(title='Top 10 Requirements for the "AI" Category',
                   xaxis_title='Requirement',
                   yaxis_title='Count')

fig2.update_layout(title='Top 10 Requirements for the "Data Science" Category',
                   xaxis_title='Requirement',
                   yaxis_title='Count')

fig3.update_layout(title='Top 10 Requirements for the "Big Data" Category',
                   xaxis_title='Requirement',
                   yaxis_title='Count')

fig1.show()
fig2.show()
fig3.show()


#### insertion des donnees

##### preparation des tables

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2883 entries, 0 to 2882
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Company                    2883 non-null   object 
 1   Job Title                  2883 non-null   object 
 2   Location                   2883 non-null   object 
 3   Job Type                   2883 non-null   object 
 4   Experience level           2883 non-null   object 
 5   Salary                     2883 non-null   float64
 6   Requirment of the company  2883 non-null   object 
 7   Facilities                 2883 non-null   object 
 8   Job Title Extracted        2883 non-null   object 
 9   Country                    2737 non-null   object 
 10  Salary Category            2883 non-null   object 
 11  Remote                     2883 non-null   bool   
 12  Hybrid                     2883 non-null   bool   
 13  Local                      2883 non-null   bool 

In [106]:
df = df.rename(columns={'Company': 'company'})
df = df.rename(columns={'Job Title': 'job_title'})
df = df.rename(columns={'Location': 'location'})
df = df.rename(columns={'Job Type': 'job_type'})
df = df.rename(columns={'Experience level': 'experience_level'})
df = df.rename(columns={'Salary': 'salary'})
df = df.rename(columns={'Requirment of the company': 'requirment_of_the_company'})
df = df.rename(columns={'Facilities': 'facilities'})
df = df.rename(columns={'Job Title Extracted': 'job_title_extracted'})
df = df.rename(columns={'Country': 'country'})
df = df.rename(columns={'Salary Category': 'salary_category'})
df = df.rename(columns={'Remote': 'remote'})
df = df.rename(columns={'Hybrid': 'hybrid'})
df = df.rename(columns={'Local': 'local'})
df = df.rename(columns={'Category': 'category'})

In [107]:
import pandas as pd

# Assuming your original DataFrame is named 'df'
company = df[['company', 'location', 'country']].copy()

# Fill null values in the 'Country' column with 'Unknown'
company['country'].fillna('Unknown', inplace=True)

# Add 1 to the row index to start 'idcompany' from 1
company['idcompany'] = company.index + 1

# Reorder the columns so that 'idcompany' comes first
company = company[['idcompany', 'company', 'location', 'country']]

# Print the DataFrame
company

,idcompany,company,location,country
0,1,sgs,"richardson, tx, united states",United States
1,2,ocorian,"ebÃ¨ne, mauritius",Mauritius / Maurice
2,3,cricut,"south jordan, ut, united states",United States
3,4,bosch group,"nonantola, italy",Italia
4,5,publicis groupe,"arlington, va, united states",United States
...,...,...,...,...
2910,2911,ccri,"chantilly, virginia, united states",United States
2911,2912,publicis groupe,"new york city, united states",United States
2912,2913,doordash,"sunnyvale, ca; san francisco, ca; new york",United States
2913,2914,western digital,"biÃ±an, philippines",Philippines


In [108]:
company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2915 entries, 0 to 2914
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   idcompany  2915 non-null   int64 
 1   company    2915 non-null   object
 2   location   2915 non-null   object
 3   country    2915 non-null   object
dtypes: int64(1), object(3)
memory usage: 91.2+ KB


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2915 entries, 0 to 2914
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   company                    2915 non-null   object 
 1   job_title                  2915 non-null   object 
 2   location                   2915 non-null   object 
 3   job_type                   2915 non-null   object 
 4   experience_level           2915 non-null   object 
 5   salary                     2915 non-null   float64
 6   requirment_of_the_company  2915 non-null   object 
 7   facilities                 2915 non-null   object 
 8   job_title_extracted        2915 non-null   object 
 9   country                    2915 non-null   object 
 10  salary_category            2915 non-null   object 
 11  remote                     2915 non-null   bool   
 12  hybrid                     2915 non-null   bool   
 13  local                      2915 non-null   bool 

In [110]:
requirements_list = df['requirment_of_the_company'].str.split(',')

# Créer un nouveau DataFrame avec Requirement of the company
requirements_exploded = pd.DataFrame({
    'company': df['company'].repeat(requirements_list.str.len()),
    'requirement_of_the_company': [item for sublist in requirements_list for item in sublist]
})

requirements_df = requirements_exploded.reset_index(drop=True)

requirements_df

,company,requirement_of_the_company
0,sgs,computer science
1,sgs,data quality
2,sgs,genetics
3,sgs,mathematics
4,sgs,sas
...,...,...
16931,atb financial,data analytics
16932,atb financial,data mining
16933,atb financial,economics
16934,atb financial,engineering


In [111]:
facilities_list = df['facilities'].str.split(',')

# Créer un nouveau DataFrame avec Requirement of the company
facilities_exploded = pd.DataFrame({
    'facilities': df['facilities'].repeat(facilities_list.str.len()),
    'facilities': [item for sublist in facilities_list for item in sublist]
})

facilities_df = facilities_exploded.reset_index(drop=True)

facilities_df

,facilities
0,without Facilities
1,without Facilities
2,career development
3,without Facilities
4,flex hours
...,...
9528,parental leave
9529,career development
9530,career development
9531,startup environment


In [112]:
import pymysql
from sqlalchemy import create_engine

DB_HOST = 'localhost'
DB_USER = 'admin1'
DB_PASS = 'Youcode-1'
DB_NAME = 'emploi'

# Assuming you have a DataFrame named 'company_df' with data for the 'company' table
columns_company = ['idcompany', 'company', 'location', 'country']

# Establish a connection to the database
connection = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, db=DB_NAME)

try:
    engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')

    # Insert data into the 'company' table
    company[columns_company].to_sql(name='company', con=engine, if_exists='append', index=False)

    print("Data inserted into 'company' table successfully.")

except Exception as e:
    print(f"Error: {str(e)}")

finally:
    connection.close()


Data inserted into 'company' table successfully.


In [113]:
import pymysql
from sqlalchemy import create_engine

DB_HOST = 'localhost'
DB_USER = 'admin1'
DB_PASS = 'Youcode-1'
DB_NAME = 'emploi'

# Assuming you have a DataFrame named 'jobs_df' with data for the 'jobs' table

# Establish a connection to the database
connection = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, db=DB_NAME)

try:
    engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')

    # Insert data into the 'jobs' table
    with connection.cursor() as cursor:
        reset_auto_increment_query = "ALTER TABLE jobs AUTO_INCREMENT = 1"
        cursor.execute(reset_auto_increment_query)

        insert_jobs_query = "INSERT INTO `jobs` (`job_title_extracted`, `experience_level`, `salary`, `salary_category`, `category`, `remote`, `hybrid`, `local`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

        for index, row in df.iterrows():
            job_title_extracted = row['job_title_extracted']
            experience_level = row['experience_level']
            salary = row['salary']
            salary_category = row['salary_category']
            category = row['category']
            remote = row['remote']
            hybrid = row['hybrid']
            local = row['local']
            cursor.execute(insert_jobs_query, (job_title_extracted, experience_level, salary, salary_category, category, remote, hybrid, local))

        connection.commit()

    print("Data inserted into 'jobs' table successfully.")

except Exception as e:
    connection.rollback()
    print(f"Error: {str(e)}")

finally:
    connection.close()


Data inserted into 'jobs' table successfully.


In [114]:
import pymysql
from sqlalchemy import create_engine

DB_HOST = 'localhost'
DB_USER = 'admin1'
DB_PASS = 'Youcode-1'
DB_NAME = 'emploi'

# Assuming you have a DataFrame named 'jobs_df' with data for the 'jobs' table

# Establish a connection to the database
connection = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, db=DB_NAME)

try:
    engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')

    # Insert data into the 'jobs' table
    with connection.cursor() as cursor:
        reset_auto_increment_query = "ALTER TABLE requirment AUTO_INCREMENT = 1"
        cursor.execute(reset_auto_increment_query)

        insert_jobs_query = "INSERT INTO `requirment` (`requirement_of_the_company`) VALUES (%s)"

        for index, row in requirements_df.iterrows():
            requirement_of_the_company = row['requirement_of_the_company']
            cursor.execute(insert_jobs_query, (requirement_of_the_company))

        connection.commit()

    print("Data inserted into 'requirment' table successfully.")

except Exception as e:
    connection.rollback()
    print(f"Error: {str(e)}")

finally:
    connection.close()


Data inserted into 'requirment' table successfully.


In [115]:
import pymysql
from sqlalchemy import create_engine

DB_HOST = 'localhost'
DB_USER = 'admin1'
DB_PASS = 'Youcode-1'
DB_NAME = 'emploi'

# Assuming you have a DataFrame named 'jobs_df' with data for the 'jobs' table

# Establish a connection to the database
connection = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, db=DB_NAME)

try:
    engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')

    # Insert data into the 'jobs' table
    with connection.cursor() as cursor:
        reset_auto_increment_query = "ALTER TABLE requirment AUTO_INCREMENT = 1"
        cursor.execute(reset_auto_increment_query)

        insert_jobs_query = "INSERT INTO `facilities` (`facilities`) VALUES (%s)"

        for index, row in facilities_df.iterrows():
            facilities = row['facilities']
            cursor.execute(insert_jobs_query, (facilities))

        connection.commit()

    print("Data inserted into 'facilities' table successfully.")

except Exception as e:
    connection.rollback()
    print(f"Error: {str(e)}")

finally:
    connection.close()


Data inserted into 'facilities' table successfully.


In [116]:
import pymysql

DB_HOST = 'localhost'
DB_USER = 'admin1'
DB_PASS = 'Youcode-1'
DB_NAME = 'emploi'

def insert_company_jobs(company):
    connection = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, db=DB_NAME)

    try:
        with connection.cursor() as cursor:
            insert_query = "INSERT INTO `company_jobs` (`company_idcompany`) VALUES (%s)"

            for index, row in company.iterrows():
                company_idcompany = row['idcompany']

                cursor.execute(insert_query, (company_idcompany))

            connection.commit()

        print("company_jobs data inserted successfully.")
    except Exception as e:
        connection.rollback()
        print(f"Error: {str(e)}")



def insert_requirment_company(company):
    connection = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, db=DB_NAME)

    try:
        with connection.cursor() as cursor:
            for index, row in company.iterrows():
                company_idcompany = row['idcompany']

                sql = "INSERT INTO requirment_company (`company_idcompany`) VALUES (%s)"

                cursor.execute(sql, (company_idcompany))

            connection.commit()

        print("requirment_company data inserted successfully.")
    except Exception as e:
        connection.rollback()
        print(f"Error: {str(e)}")


def insert_facilities_company(company):
    connection = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, db=DB_NAME)

    try:
        with connection.cursor() as cursor:
            for index, row in company.iterrows():
                company_idcompany = row['idcompany']

                sql = "INSERT INTO facilities_company (`company_idcompany`) VALUES (%s)"

                cursor.execute(sql, (company_idcompany))

            connection.commit()

        print("facilities_company data inserted successfully.")
    except Exception as e:
        connection.rollback()
        print(f"Error: {str(e)}")

    finally:
        connection.close()

insert_company_jobs(company)
insert_requirment_company(company)
insert_facilities_company(company)


company_jobs data inserted successfully.
requirment_company data inserted successfully.
facilities_company data inserted successfully.
